In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

df = pd.read_excel('sales_and_eodStocks.xlsx', sheet_name = 'Sheet1') 

df['Product_ID'] = df['Product_ID'].astype(str)

result_detect_bias = df.groupby('Product_ID').agg(
    sales_count=('Product_ID', 'count'),
    total_demand=('Sales', 'sum'),
    first_stock=('EndOfDayStock', 'first'),
    first_sale=('Sales', 'first')
).reset_index()
result_detect_bias['total_forecast'] = result_detect_bias['first_stock'] + result_detect_bias['first_sale']
result_detect_bias['bias'] = (result_detect_bias['total_forecast'] - result_detect_bias['total_demand']) / result_detect_bias['sales_count']

#formula bias fortata: [(forecast[0] - demand[0]) + (forecast[1] - demand[1]) + ... (forecast[n-1] - demand[n-1])] / n
#[sum(forecast) - sum(demand)] / n, deci calculam bias-ul si RMSE pentru cei 2 ani

result_detect_bias['situation'] = pd.cut(result_detect_bias['bias'], bins=[float('-inf'), 0, 3, float('inf')],
                                labels=['out-of-stock', 'good amount', 'overstock'], right=False)

result_detect_bias = result_detect_bias[result_detect_bias['situation'] == "good amount"]
result_detect_bias_sorted = result_detect_bias.sort_values(by='total_demand', ascending=False)
result_detect_bias_sorted.to_json('predict_with_bias.json', orient='records', lines=True)
